ML model for Formula One.

## Part 1: Database and Jupyter Notebook Set Up

#### Part 1 - Step 1 
Import the data provided in the `last_five_years_results_df.json` file from your Mongo Compass into a new database called `formula_one` and a collection called `results`.

#### Part 1 - Step 2
Import required dependencies.

In [46]:
from pymongo import MongoClient
from pprint  import pprint
import pandas as pd

#### Part 1 - Step 3
Create an instance of the Mongo Client.

In [13]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

#### Part 1 - Step 4
- List the databases you have in MongoDB. Confirm that uk_food is listed.
- List the collection(s) in the database to ensure that establishments is there.
- Find and display one document in the establishments collection using find_one and display with pprint.
- Assign the establishments collection to a variable to prepare the collection for use.

In [14]:
# confirm that our new database was created
# The database called uk_food, must appear in the list.
# Ohter databases may exist as leftover from other exercises/projects.
print(mongo.list_database_names())

['F1', 'admin', 'config', 'local', 'met', 'uk_food']


In [15]:
# assign the uk_food database to a variable name
db = mongo['F1']

In [16]:
# review the collections in our new database
# The name of the collection must appear in the list.
print(db.list_collection_names())

['Drivers', 'Teams', 'Results']


In [17]:
# review a document in the establishments collection
pprint(db.Results.find_one())

{'': 0,
 'Driver': 'Sebastian Vettel',
 'Laps': 58,
 'No': 5,
 'Points': 25.0,
 'Position': 1,
 'Starting Grid': 3.0,
 'Team': 'Ferrari',
 'Time/Retired': '29:33.3',
 'Track': 'Australia',
 'Year': 2018,
 '_id': ObjectId('6515c3e0a6ad92a32b7c11d8')}


In [9]:
# assign the collection to a variable
Results = db['Results']

## Part 2: Update the Database

The magazine editors have some requested modifications for the database
before you can perform any queries or analysis for them. Make the following
changes to the establishments collection:

#### Part 2 - Step 1
- An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you 
  to  include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [51]:
# Check that the new restaurant was inserted
# Filter results by name
query  = ({"Position" : {"$gt": 0}})
fields = {'Position'  : 1,
          'Driver': 1,
          'Starting Grid': 1,
          'Laps': 1,
          'Time/Retired': 1,
          'Points': 1}
results = Results.find(query,fields)
#for result in results:
#    pprint(result.get('Driver'))
results_df = pd.DataFrame(list(results))
print(results_df.head())


                        _id  Position            Driver  Starting Grid  Laps  \
0  6515c3e0a6ad92a32b7c11d8         1  Sebastian Vettel            3.0    58   
1  6515c3e0a6ad92a32b7c11d9         2    Lewis Hamilton            1.0    58   
2  6515c3e0a6ad92a32b7c11da         3  Kimi RaikkAAPnen            2.0    58   
3  6515c3e0a6ad92a32b7c11db         4  Daniel Ricciardo            8.0    58   
4  6515c3e0a6ad92a32b7c11dc         5   Fernando Alonso           10.0    58   

  Time/Retired  Points  
0      29:33.3    25.0  
1        5.036    18.0  
2        6.309    15.0  
3        7.069    12.0  
4       27.886    10.0  


#### Part 2 - Step 2
Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [46]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType':'Restaurant/Cafe/Canteen'}
fields = { 'BusinessTypeID' : 1, 'BusinessType': 1 }
result = establishments.find(query, fields)
x=result[0]
bussinesstypeid = x['BusinessTypeID']
print(bussinesstypeid)

1


#### Part 2 - Step 3
Update the new restaurant with the `BusinessTypeID` you found.

In [47]:
# Update the new restaurant with the correct BusinessTypeID

establishments.update_one( {'BusinessName': 'Penang Flavours'},  { '$set': { 'BusinessTypeID': f'{bussinesstypeid}'} })

In [48]:
# Confirm that the new restaurant was updated
# Check that the new restaurant was inserted
# Filter results by name
# Find how many documents have LocalAuthorityName as "Dover"
query = {'BusinessName': 'Penang Flavours'}
query = ({"age":{"$gt":"26"}})  
fields = { 'BusinessName':1, 'BusinessTypeID' : 1, 'BusinessType': 1 }
results = establishments.find(query,fields)
for result in results:
    pprint(result)

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 '_id': ObjectId('64a86993555a3ea5d685feba')}


#### Part 2 - Step 4

The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [49]:
# Find how many documents have LocalAuthorityName as "Dover"
# Delete all documents where LocalAuthorityName is "Dover"
# Check if any remaining documents include Dover
query         = {'LocalAuthorityName': 'Dover'}
fields        = { 'LocalAuthorityName' : 1, 'BusinessType': 1 }
AllLocalQty   = establishments.count_documents({})
DoverLocalQty = establishments.count_documents(query)
AfterDelete   = AllLocalQty - DoverLocalQty
establishments.delete_many( {'LocalAuthorityName': 'Dover'} )
Confirmation = establishments.count_documents({})
print(F'All bussinesses           : {AllLocalQty}')
print(F'Dover Bussinesses         : {DoverLocalQty}')
print(F'After Delete Dover        : {AfterDelete}')
AllLocalQty   = establishments.count_documents({})
print(F'After Delete Confirmation : {AllLocalQty}')

All bussinesses           : 39780
Dover Bussinesses         : 994
After Delete Dover        : 38786
After Delete Confirmation : 38786


In [50]:
# Check if any remaining documents include Dover
query         = {'LocalAuthorityName': 'Dover'}
AllLocalQty   = establishments.find(query)
# None should appear.

In [51]:
# Check that other documents remain with 'find_one'
est = establishments.find_one()
pprint(est)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a8697cdd2ceeaae657cdd5'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

#### Part 2 - Step 5
Some of the number values are stored as strings, when they should be stored as numbers.
Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [52]:
# Change the data type from String to Decimal for longitude and latitude
# Using Double would ensure a better precision, but the requirement is to use decimal to ensure a 
# better precision for longitude and latitude
establishments.update_many({}, 
                           [ {'$set': { "geocode.latitude"  : { '$toDecimal': "$geocode.latitude" },
                                        "geocode.longitude" : { '$toDecimal': "$geocode.longitude" }
                                      }
                             } ]
                          )

est = establishments.find_one()
pprint(est)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a8697cdd2ceeaae657cdd5'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel

Use `update_many` to convert `RatingValue` to integer numbers.

In [53]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
est = establishments.find_one()
pprint(est)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a8697cdd2ceeaae657cdd5'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel

In [54]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, 
                           [ {'$set': { "RatingValue"  : { '$toInt': "$RatingValue" }
                                      }
                             } ]
                          )
est = establishments.find_one()
pprint(est)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64a8697cdd2ceeaae657cdd5'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':

In [55]:
# Check that the coordinates and rating value are now numbers
est = establishments.find_one({},{ 'geocode.latitude': 1, 'geocode.longitude': 1, 'RatingValue' : 1 })
pprint(est)

{'RatingValue': 5,
 '_id': ObjectId('64a8697cdd2ceeaae657cdd5'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')}}
